## 0. Imports, libraries and rusable functions

In [18]:
# Standard Library Imports
import ast
import copy
import csv
import json
import math
import os
import re
import time
import warnings
import logging
import random
import collections
from collections import Counter
from typing import List, Tuple, Optional
from IPython.display import HTML, display
import math
import time
from unidecode import unidecode


# Data Handling Libraries
import numpy as np
import pandas as pd
import csv
from torch.utils.data import random_split
import datasets
from datasets import ClassLabel, Sequence

# Data Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
# import scikitplot as skplt  # Uncomment if scikit-plot is installed and needed

# Machine Learning: Model Preparation
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
from sklearn.model_selection import cross_val_score, cross_validate, KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler

# Machine Learning: Models and Frameworks
import tensorflow as tf
import torch
import evaluate
import xgboost
import wandb
from xgboost import plot_importance  # Uncomment if xgboost importance plot is required


# NLP and Transformers
from transformers import (AdamW, AutoModelForSequenceClassification, AutoModelForQuestionAnswering,
                          AutoTokenizer, CamembertForSequenceClassification, DistilBertConfig,
                          DistilBertForSequenceClassification, DistilBertModel, EarlyStoppingCallback,
                          get_linear_schedule_with_warmup, RobertaForSequenceClassification, EvalPrediction,
                          Trainer, TrainerCallback, TrainingArguments, XLMRobertaForSequenceClassification,
                         DefaultDataCollator, BertForQuestionAnswering, DataCollatorWithPadding, PreTrainedTokenizerFast,
                         default_data_collator, is_torch_xla_available)
from datasets import Dataset, DatasetDict, load_dataset
from transformers.trainer_utils import PredictionOutput, speed_metrics

# Experiment Tracking and Optimization Utilities
import optuna
from optuna.trial import TrialState
# import wandb  # Uncomment if using Weights & Biases for experiment tracking

# Progress Bar Utilities
from tqdm.notebook import tqdm


In [20]:
#wandb.login(key='8f7092f0fdaf14add2b4cc07cb0e740080cdd8e7')
wandb.login()

wandb: Currently logged in as: mzak071 (COMPSCI714). Use `wandb login --relogin` to force relogin


True

In [21]:
class LoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

GPU: NVIDIA GeForce RTX 4070 Ti SUPER is available.
cuda


## 1. Global Variables

In [23]:
## Arguments and global vriables
global_doc_stride = 128
pretrained_model_name = "distilbert-base-cased"
normalized_model_name = pretrained_model_name.replace("/", "-")
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
assert isinstance( tokenizer, PreTrainedTokenizerFast )
data_collator = DefaultDataCollator()
max_seq_length = tokenizer.model_max_length
version_2_with_negative = True
no_answer_threshold = 0.0166
right_padding = tokenizer.padding_side == 'right'
global_counter = 0
traing_answer_mismatches = []
logger = logging.getLogger(__name__)
global train_dataset
global eval_dataset
global cleaned_training_dataset

## 1. Q&A Reusable Functions

In [25]:
### Compute_metrics function for Question and Answering problem is different to classification, more preocessing required.
metric = evaluate.load("squad_v2")

def compute_metrics(p: EvalPrediction):
        return metric.compute(predictions=p.predictions, references=p.label_ids)

In [28]:
"""
A subclass of `Trainer` specific to Question-Answering tasks
"""

if is_torch_xla_available():
    import torch_xla.core.xla_model as xm
    import torch_xla.debug.metrics as met


class QuestionAnsweringTrainer(Trainer):
    def __init__(self, *args, eval_examples=None, post_process_function=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.eval_examples = eval_examples
        self.post_process_function = post_process_function

    def evaluate(self, eval_dataset=None, eval_examples=None, ignore_keys=None, metric_key_prefix: str = "eval"):
        eval_dataset = self.eval_dataset if eval_dataset is None else eval_dataset
        eval_dataloader = self.get_eval_dataloader(eval_dataset)
        eval_examples = self.eval_examples if eval_examples is None else eval_examples

        # Temporarily disable metric computation, we will do it in the loop here.
        compute_metrics = self.compute_metrics
        self.compute_metrics = None
        eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else self.evaluation_loop
        start_time = time.time()
        try:
            output = eval_loop(
                eval_dataloader,
                description="Evaluation",
                # No point gathering the predictions if there are no metrics, otherwise we defer to
                # self.args.prediction_loss_only
                prediction_loss_only=True if compute_metrics is None else None,
                ignore_keys=ignore_keys,
                metric_key_prefix=metric_key_prefix,
            )
        finally:
            self.compute_metrics = compute_metrics
        total_batch_size = self.args.eval_batch_size * self.args.world_size
        if f"{metric_key_prefix}_jit_compilation_time" in output.metrics:
            start_time += output.metrics[f"{metric_key_prefix}_jit_compilation_time"]
        output.metrics.update(
            speed_metrics(
                metric_key_prefix,
                start_time,
                num_samples=output.num_samples,
                num_steps=math.ceil(output.num_samples / total_batch_size),
            )
        )
        if self.post_process_function is not None and self.compute_metrics is not None and self.args.should_save:
            # Only the main node write the results by default
            eval_preds = self.post_process_function(eval_examples, eval_dataset, output.predictions)
            metrics = self.compute_metrics(eval_preds)

            # Prefix all keys with metric_key_prefix + '_'
            for key in list(metrics.keys()):
                if not key.startswith(f"{metric_key_prefix}_"):
                    metrics[f"{metric_key_prefix}_{key}"] = metrics.pop(key)
            metrics.update(output.metrics)
        else:
            metrics = output.metrics

        if self.args.should_log:
            # Only the main node log the results by default
            self.log(metrics)

        if self.args.tpu_metrics_debug or self.args.debug:
            # tpu-comment: Logging debug metrics for PyTorch/XLA (compile, execute times, ops, etc.)
            xm.master_print(met.metrics_report())

        self.control = self.callback_handler.on_evaluate(self.args, self.state, self.control, metrics)
        return metrics

    def predict(self, predict_dataset, predict_examples, ignore_keys=None, metric_key_prefix: str = "test"):
        predict_dataloader = self.get_test_dataloader(predict_dataset)

        # Temporarily disable metric computation, we will do it in the loop here.
        compute_metrics = self.compute_metrics
        self.compute_metrics = None
        eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else self.evaluation_loop
        start_time = time.time()
        try:
            output = eval_loop(
                predict_dataloader,
                description="Prediction",
                # No point gathering the predictions if there are no metrics, otherwise we defer to
                # self.args.prediction_loss_only
                prediction_loss_only=True if compute_metrics is None else None,
                ignore_keys=ignore_keys,
                metric_key_prefix=metric_key_prefix,
            )
        finally:
            self.compute_metrics = compute_metrics
        total_batch_size = self.args.eval_batch_size * self.args.world_size
        if f"{metric_key_prefix}_jit_compilation_time" in output.metrics:
            start_time += output.metrics[f"{metric_key_prefix}_jit_compilation_time"]
        output.metrics.update(
            speed_metrics(
                metric_key_prefix,
                start_time,
                num_samples=output.num_samples,
                num_steps=math.ceil(output.num_samples / total_batch_size),
            )
        )

        if self.post_process_function is None or self.compute_metrics is None:
            return output

        predictions = self.post_process_function(predict_examples, predict_dataset, output.predictions, "predict")
        metrics = self.compute_metrics(predictions)

        # Prefix all keys with metric_key_prefix + '_'
        for key in list(metrics.keys()):
            if not key.startswith(f"{metric_key_prefix}_"):
                metrics[f"{metric_key_prefix}_{key}"] = metrics.pop(key)
        metrics.update(output.metrics)
        return PredictionOutput(predictions=predictions.predictions, label_ids=predictions.label_ids, metrics=metrics)

In [30]:
def prepare_validation_features(examples):
  # Some of the questions have lots of whitespace on the left, which is not useful and will make the
  # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
  # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]
  # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
  # in one example possible giving several features when a context is long, each of those features having a
  # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
      examples["question"],
      examples["context"],
      truncation="only_second" if right_padding else "only_first",
      max_length=max_seq_length,
      stride=global_doc_stride,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length",
  )

  # Since one example might give us several features if it has a long context, we need a map from a feature to
  # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

  # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
  # corresponding example_id and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
      # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 0

      # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

      # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
      # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
      ]

    return tokenized_examples

In [31]:
# Validation preprocessing
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if right_padding else "context"],
        examples["context" if right_padding else "question"],
        truncation="only_second" if right_padding else "only_first",
        max_length=max_seq_length,
        stride=global_doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
    # corresponding example_id and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if right_padding else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [32]:
# Post-processing:
def post_processing_function(examples, features, predictions, stage="eval"):
    # Post-processing: we match the start logits and end logits to answers in the original context.
    predictions = postprocess_qa_predictions(
        examples=examples,
        features=features,
        predictions=predictions,
        version_2_with_negative=version_2_with_negative, #If true, some of the examples do not have an answer.
        n_best_size=20, #The total number of n-best predictions to generate when looking for an answer.
        max_answer_length=30, #The maximum length of an answer that can be generated. This is needed because the start and end predictions are not conditioned on one another.
        null_score_diff_threshold=no_answer_threshold, #The threshold used to select the null answer: if the best answer has a score that is less than the score of the null answer minus this threshold, the null answer is selected for this example. Only useful when `version_2_with_negative=True`.
        output_dir= None,
        log_level=logging.WARNING,
        prefix=stage,
    )
    # Format the result to the format the metric expects.
    if version_2_with_negative:
        formatted_predictions = [
                {"id": str(k), "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()
            ]
    else:
        formatted_predictions = [{"id": str(k), "prediction_text": v} for k, v in predictions.items()]
    
    
    references = [{"id": str(ex["id"]), "answers": ex["answers"]} for ex in examples]
    return EvalPrediction(predictions=formatted_predictions, label_ids=references)

In [33]:
def postprocess_qa_predictions(
    examples,
    features,
    predictions: Tuple[np.ndarray, np.ndarray],
    version_2_with_negative: bool = False,
    n_best_size: int = 20,
    max_answer_length: int = 30,
    null_score_diff_threshold: float = 0.0,
    output_dir: Optional[str] = None,
    prefix: Optional[str] = None,
    log_level: Optional[int] = logging.WARNING,
):
    """
    Post-processes the predictions of a question-answering model to convert them to answers that are substrings of the
    original contexts. This is the base postprocessing functions for models that only return start and end logits.

    Args:
        examples: The non-preprocessed dataset (see the main script for more information).
        features: The processed dataset (see the main script for more information).
        predictions (:obj:`Tuple[np.ndarray, np.ndarray]`):
            The predictions of the model: two arrays containing the start logits and the end logits respectively. Its
            first dimension must match the number of elements of :obj:`features`.
        version_2_with_negative (:obj:`bool`, `optional`, defaults to :obj:`False`):
            Whether or not the underlying dataset contains examples with no answers.
        n_best_size (:obj:`int`, `optional`, defaults to 20):
            The total number of n-best predictions to generate when looking for an answer.
        max_answer_length (:obj:`int`, `optional`, defaults to 30):
            The maximum length of an answer that can be generated. This is needed because the start and end predictions
            are not conditioned on one another.
        null_score_diff_threshold (:obj:`float`, `optional`, defaults to 0):
            The threshold used to select the null answer: if the best answer has a score that is less than the score of
            the null answer minus this threshold, the null answer is selected for this example (note that the score of
            the null answer for an example giving several features is the minimum of the scores for the null answer on
            each feature: all features must be aligned on the fact they `want` to predict a null answer).

            Only useful when :obj:`version_2_with_negative` is :obj:`True`.
        output_dir (:obj:`str`, `optional`):
            If provided, the dictionaries of predictions, n_best predictions (with their scores and logits) and, if
            :obj:`version_2_with_negative=True`, the dictionary of the scores differences between best and null
            answers, are saved in `output_dir`.
        prefix (:obj:`str`, `optional`):
            If provided, the dictionaries mentioned above are saved with `prefix` added to their names.
        log_level (:obj:`int`, `optional`, defaults to ``logging.WARNING``):
            ``logging`` log level (e.g., ``logging.WARNING``)
    """
    if len(predictions) != 2:
        raise ValueError("`predictions` should be a tuple with two elements (start_logits, end_logits).")
    all_start_logits, all_end_logits = predictions

    if len(predictions[0]) != len(features):
        raise ValueError(f"Got {len(predictions[0])} predictions and {len(features)} features.")

    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    all_predictions = collections.OrderedDict()
    all_nbest_json = collections.OrderedDict()
    if version_2_with_negative:
        scores_diff_json = collections.OrderedDict()

    # Logging.
    logger.setLevel(log_level)
    logger.info(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_prediction = None
        prelim_predictions = []

        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]
            # Optional `token_is_max_context`, if provided we will remove answers that do not have the maximum context
            # available in the current feature.
            token_is_max_context = features[feature_index].get("token_is_max_context", None)

            # Update minimum null prediction.
            feature_null_score = start_logits[0] + end_logits[0]
            if min_null_prediction is None or min_null_prediction["score"] > feature_null_score:
                min_null_prediction = {
                    "offsets": (0, 0),
                    "score": feature_null_score,
                    "start_logit": start_logits[0],
                    "end_logit": end_logits[0],
                }

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or len(offset_mapping[start_index]) < 2
                        or offset_mapping[end_index] is None
                        or len(offset_mapping[end_index]) < 2
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue
                    # Don't consider answer that don't have the maximum context available (if such information is
                    # provided).
                    if token_is_max_context is not None and not token_is_max_context.get(str(start_index), False):
                        continue

                    prelim_predictions.append(
                        {
                            "offsets": (offset_mapping[start_index][0], offset_mapping[end_index][1]),
                            "score": start_logits[start_index] + end_logits[end_index],
                            "start_logit": start_logits[start_index],
                            "end_logit": end_logits[end_index],
                        }
                    )
        if version_2_with_negative and min_null_prediction is not None:
            # Add the minimum null prediction
            prelim_predictions.append(min_null_prediction)
            null_score = min_null_prediction["score"]

        # Only keep the best `n_best_size` predictions.
        predictions = sorted(prelim_predictions, key=lambda x: x["score"], reverse=True)[:n_best_size]

        # Add back the minimum null prediction if it was removed because of its low score.
        if (
            version_2_with_negative
            and min_null_prediction is not None
            and not any(p["offsets"] == (0, 0) for p in predictions)
        ):
            predictions.append(min_null_prediction)

        # Use the offsets to gather the answer text in the original context.
        context = example["context"]
        for pred in predictions:
            offsets = pred.pop("offsets")
            pred["text"] = context[offsets[0] : offsets[1]]

        # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
        # failure.
        if len(predictions) == 0 or (len(predictions) == 1 and predictions[0]["text"] == ""):
            predictions.insert(0, {"text": "empty", "start_logit": 0.0, "end_logit": 0.0, "score": 0.0})

        # Compute the softmax of all scores (we do it with numpy to stay independent from torch/tf in this file, using
        # the LogSumExp trick).
        scores = np.array([pred.pop("score") for pred in predictions])
        exp_scores = np.exp(scores - np.max(scores))
        probs = exp_scores / exp_scores.sum()

        # Include the probabilities in our predictions.
        for prob, pred in zip(probs, predictions):
            pred["probability"] = prob

        # Pick the best prediction. If the null answer is not possible, this is easy.
        if not version_2_with_negative:
            all_predictions[example["id"]] = predictions[0]["text"]
        else:
            # Otherwise we first need to find the best non-empty prediction.
            i = 0
            while predictions[i]["text"] == "":
                i += 1
            best_non_null_pred = predictions[i]

            # Then we compare to the null prediction using the threshold.
            score_diff = null_score - best_non_null_pred["start_logit"] - best_non_null_pred["end_logit"]
            scores_diff_json[example["id"]] = float(score_diff)  # To be JSON-serializable.
            if score_diff > null_score_diff_threshold:
                all_predictions[example["id"]] = ""
            else:
                all_predictions[example["id"]] = best_non_null_pred["text"]

        # Make `predictions` JSON-serializable by casting np.float back to float.
        all_nbest_json[example["id"]] = [
            {k: (float(v) if isinstance(v, (np.float16, np.float32, np.float64)) else v) for k, v in pred.items()}
            for pred in predictions
        ]

    # If we have an output_dir, let's save all those dicts.
    if output_dir is not None:
        if not os.path.isdir(output_dir):
            raise EnvironmentError(f"{output_dir} is not a directory.")

        prediction_file = os.path.join(
            output_dir, "predictions.json" if prefix is None else f"{prefix}_predictions.json"
        )
        nbest_file = os.path.join(
            output_dir, "nbest_predictions.json" if prefix is None else f"{prefix}_nbest_predictions.json"
        )
        if version_2_with_negative:
            null_odds_file = os.path.join(
                output_dir, "null_odds.json" if prefix is None else f"{prefix}_null_odds.json"
            )

        logger.info(f"Saving predictions to {prediction_file}.")
        with open(prediction_file, "w") as writer:
            writer.write(json.dumps(all_predictions, indent=4) + "\n")
        logger.info(f"Saving nbest_preds to {nbest_file}.")
        with open(nbest_file, "w") as writer:
            writer.write(json.dumps(all_nbest_json, indent=4) + "\n")
        if version_2_with_negative:
            logger.info(f"Saving null_odds to {null_odds_file}.")
            with open(null_odds_file, "w") as writer:
                writer.write(json.dumps(scores_diff_json, indent=4) + "\n")

    return all_predictions

In [34]:
# List of questions to debug
debug_questions = [
    "What ideology was sponsored at the Ming court?",
    "Who stopped their trips to Ming China?",
    "Another question of interest"
]

# Training preprocessing
def preprocess_function(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if right_padding else "context"],
        examples["context" if right_padding else "question"],
        truncation="only_second" if right_padding else "only_first",
        max_length=max_seq_length,
        stride=global_doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]        
        question = examples["question"][sample_index]
        example_id = examples["id"][sample_index]
        
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if right_padding else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if right_padding else 0):
                token_end_index -= 1
            
            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Increase the end index here to make sure it includes the last character
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                safe_start_index = max(0, token_start_index - 1)
                tokenized_examples["start_positions"].append(safe_start_index)

                # Adjust this line to ensure that the end character is included
                # Decrement token_end_index to point exactly one past the end of the answer text
                while token_end_index >= 0 and offsets[token_end_index][1] > end_char:
                    token_end_index -= 1
                safe_end_index = min(len(input_ids) - 1, token_end_index + 1)
                tokenized_examples["end_positions"].append(safe_end_index)  # Adjust to +2 if needed
                
        # Debugging code
        global global_counter
        global traing_answer_mismatches
        #if question in debug_questions:
        if len(answers["answer_start"]) > 0:
            # Ensure indices are within bounds
            #safe_start_index = max(0, token_start_index - 1)
            #safe_end_index = min(len(input_ids) - 1, token_end_index + 1)

            # Decode the answer using safe indices
            decoded_answer = tokenizer.decode(input_ids[safe_start_index:safe_end_index])
            actual_answer = answers["text"][0] if answers["text"] else "No answer provided"
            normalized_actual_answer = actual_answer.lower().replace(" ", "")
            normalized_decoded_answer = decoded_answer.lower().replace(" ", "")
            if normalized_decoded_answer != normalized_actual_answer:
                if global_counter == 0: print('Mismatch Found')
                global_counter += 1 
                traing_answer_mismatches.append({
                    'TokenizedId': i,
                    'ID': example_id,
                    'Question': question,
                    'Offsets': offsets,
                    'Input IDs':input_ids,
                    'Tokenized Text': tokenizer.decode(input_ids),
                    'Char start/end index': f"{start_char} / {end_char}",
                    'Token start/end index':f"{token_start_index} / {token_end_index}",
                    'Decoded Answer': decoded_answer,
                    'Actual Answer': actual_answer,                    
                })    
    return tokenized_examples

## 2. Data Preparation

In [43]:
# Load the full dataset
full_squad_dataset = load_dataset("squad_v2")

In [44]:
full_squad_dataset['train'].column_names

['id', 'title', 'context', 'question', 'answers']

In [45]:
full_squad_dataset['validation'].column_names

['id', 'title', 'context', 'question', 'answers']

In [46]:
Use_Only_Sample = False
Sample_Size = 10000

if Use_Only_Sample:
    # Load a sample portion of the dataset
    squad_raw = datasets.DatasetDict({
        'train': full_squad_dataset['train'].select(range(0, Sample_Size)),
        'validation': full_squad_dataset['validation'].select(range(0, int(Sample_Size * 0.2)))
    })
else:
    # Load the full dataset
    squad_raw = datasets.DatasetDict({
        'train': full_squad_dataset['train'],
        'validation': full_squad_dataset['validation'],    
    })
    
# Display the sizes of the splits to confirm
print("Train set size:", len(squad_raw['train']))
print("Validation set size:", len(squad_raw['validation']))
squad_raw

Train set size: 130319
Validation set size: 11873


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [47]:
## Preprocess the Training Dataset To find Missmatches
global_counter = 0
traing_answer_mismatches = []

train_dataset = squad_raw['train'].map(
                preprocess_function,
                batched=True,
                remove_columns=squad_raw["train"].column_names,
                desc="Running tokenizer on train dataset",
                load_from_cache_file=False,  # Disable caching
            )


Running tokenizer on train dataset:   0%|          | 0/130319 [00:00<?, ? examples/s]

Mismatch Found


In [48]:
# Analyse Missmatches
mismatches_df = pd.DataFrame(traing_answer_mismatches)
print('Total number of miss matches:',global_counter)
display(mismatches_df.head(10))  

Total number of miss matches: 376


,TokenizedId,ID,Question,Offsets,Input IDs,Tokenized Text,Char start/end index,Token start/end index,Decoded Answer,Actual Answer
0,92,56bf7e603aeaaa14008c9681,What event caused Beyonce's depression?,"[(0, 0), (0, 4), (5, 10), (11, 17), (18, 21), ...","[101, 1327, 1856, 2416, 24896, 1320, 2093, 112...",[CLS] What event caused Beyonce's depression? ...,194 / 222,49 / 53,split with Luckett and Rob,split with Luckett and Rober
1,203,56be973d3aeaaa14008c9123,How many number one singles did Beyonce now ha...,"[(0, 0), (0, 3), (4, 8), (9, 15), (16, 19), (2...","[101, 1731, 1242, 1295, 1141, 3896, 1225, 2489...",[CLS] How many number one singles did Beyonce ...,457 / 460,131 / 129,,six
2,860,56cbdea66d243a140015edae,At what age did Frédéric start giving public c...,"[(0, 0), (0, 2), (3, 7), (8, 11), (12, 15), (1...","[101, 1335, 1184, 1425, 1225, 13359, 2744, 118...",[CLS] At what age did Frédéric start giving pu...,391 / 392,110 / 109,1817,7
3,865,56cf609aaab44d1400b89187,At what age did Chopin start playing publicly?,"[(0, 0), (0, 2), (3, 7), (8, 11), (12, 15), (1...","[101, 1335, 1184, 1425, 1225, 22964, 6709, 183...",[CLS] At what age did Chopin start playing pub...,391 / 392,106 / 105,1817,7
4,870,56d20a6ae7d4791d0090261a,How old was Chopin when he began to perform fo...,"[(0, 0), (0, 3), (4, 7), (8, 11), (12, 15), (1...","[101, 1731, 1385, 1108, 22964, 6709, 1165, 111...",[CLS] How old was Chopin when he began to perf...,391 / 392,110 / 109,1817,7
5,192,56d3ac8e2ccc5a1400d82e1b,"What style is the fictionalized ""Chopin"" in?","[(0, 0), (0, 4), (5, 10), (11, 13), (14, 17), ...","[101, 1327, 1947, 1110, 1103, 6725, 2200, 107,...","[CLS] What style is the fictionalized "" Chopin...",85 / 90,35 / 33,,opera
6,513,56ccde7862d2951400fa64d9,Who had a large amount of contacts with china ...,"[(0, 0), (0, 3), (4, 7), (8, 9), (10, 15), (16...","[101, 2627, 1125, 170, 1415, 2971, 1104, 10492...",[CLS] Who had a large amount of contacts with ...,193 / 198,64 / 62,,Tibet
7,574,56cd682162d2951400fa658e,What does Yiu Yung-chin claim the Ming did not...,"[(0, 0), (0, 4), (5, 9), (10, 12), (12, 13), (...","[101, 1327, 1674, 14141, 1358, 27413, 1403, 11...",[CLS] What does Yiu Yung - chin claim the Ming...,55 / 60,28 / 26,,Tibet
8,719,56ce4a58aab44d1400b88668,When were the Rinbung princes overthrown?,"[(0, 0), (0, 4), (5, 9), (10, 13), (14, 15), (...","[101, 1332, 1127, 1103, 155, 1394, 7925, 2118,...",[CLS] When were the Rinbung princes overthrown...,3 / 7,13 / 179,##tymie,1565
9,721,56ce4a58aab44d1400b88669,Who took control of the whole of Central Tibet?,"[(0, 0), (0, 3), (4, 8), (9, 16), (17, 19), (2...","[101, 2627, 1261, 1654, 1104, 1103, 2006, 1104...",[CLS] Who took control of the whole of Central...,247 / 269,12 / 177,", the Dalai Lama's chief",Karma Phuntsok Namgyal


In [49]:
# List of specific IDs to inspect
debug_ids = [
    "56be8c8a3aeaaa14008c90ab",       
]

# Define a function to filter examples by ID and stop processing when all have been found
def print_specific_records(dataset):
    found_ids = set()  # To track IDs that have been printed
    for example in dataset:
        if example['id'] in debug_ids:
            print(example)
            found_ids.add(example['id'])
            if found_ids == set(debug_ids):
                break  # Stop processing as all requested records have been printed

# Apply the function to the full dataset (assuming the dataset is loaded and named full_squad_dataset)
print_specific_records(full_squad_dataset['train'])


{'id': '56be8c8a3aeaaa14008c90ab', 'title': 'Beyoncé', 'context': 'The group changed their name to Destiny\'s Child in 1996, based upon a passage in the Book of Isaiah. In 1997, Destiny\'s Child released their major label debut song "Killing Time" on the soundtrack to the 1997 film, Men in Black. The following year, the group released their self-titled debut album, scoring their first major hit "No, No, No". The album established the group as a viable act in the music industry, with moderate sales and winning the group three Soul Train Lady of Soul Awards for Best R&B/Soul Album of the Year, Best R&B/Soul or Rap New Artist, and Best R&B/Soul Single for "No, No, No". The group released their multi-platinum second album The Writing\'s on the Wall in 1999. The record features some of the group\'s most widely known songs such as "Bills, Bills, Bills", the group\'s first number-one single, "Jumpin\' Jumpin\'" and "Say My Name", which became their most successful song at the time, and would 

In [50]:
# Ensure the mismatches DataFrame is created from non-empty data
if traing_answer_mismatches:
    mismatches_df = pd.DataFrame(traing_answer_mismatches)
    print('Total number of mismatches:', len(mismatches_df))
else:
    print("No mismatches to display. The list is empty.")

def remove_mismatches(dataset, mismatch_ids):
    """
    Filters the dataset to exclude mismatched entries.

    Args:
        dataset (Dataset): The dataset from which to remove mismatched examples.
        mismatch_ids (set): A set of example IDs that have mismatches.

    Returns:
        Dataset: A dataset with mismatched examples removed.
    """
    # Use a set for faster look-up times
    filtered_dataset = dataset.filter(lambda example: example['id'] not in mismatch_ids)
    return filtered_dataset

# Check if mismatches DataFrame exists and is not empty before removing mismatches
if 'traing_answer_mismatches' in locals() and not mismatches_df.empty:
    mismatch_ids = set(mismatches_df['ID'])
    cleaned_training_dataset = remove_mismatches(squad_raw['train'], mismatch_ids)
    print("Original dataset size:", len(squad_raw['train']))
    print("Cleaned dataset size:", len(cleaned_training_dataset))
else:
    print("No mismatches found or mismatch data is empty. No filtering applied.")
    cleaned_training_dataset = squad_raw['train']


Total number of mismatches: 376
Original dataset size: 130319
Cleaned dataset size: 129944


In [51]:

# Preprocessing the datasets
train_dataset = cleaned_training_dataset.map(
                preprocess_function,
                batched=True,
                remove_columns=squad_raw["train"].column_names,
                desc="Running tokenizer on train dataset",
                load_from_cache_file=False,  # Disable caching
            )
eval_dataset = squad_raw['validation'].map(
                prepare_validation_features,
                batched=True,
                remove_columns=squad_raw["train"].column_names,
                desc="Running tokenizer on validation dataset",
                load_from_cache_file=False,  # Disable caching
            )
eval_examples =  squad_raw["validation"]


Running tokenizer on train dataset:   0%|          | 0/129944 [00:00<?, ? examples/s]

Running tokenizer on validation dataset:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [52]:
# Display the sizes of the splits to confirm
print("Train set size:", len(train_dataset))
print("Validation set size:", len(eval_dataset))
print("Validation Examples size:", len(eval_examples))

Train set size: 130049
Validation set size: 11974
Validation Examples size: 11873


## 3. Initial Tuning of DistilBERT Model


For Intial training we used the fine-tuned parameters of the model https://huggingface.co/deepset/roberta-base-squad2

In [81]:
no_answer_threshold = 0.0
model = AutoModelForQuestionAnswering.from_pretrained(pretrained_model_name)
training_args = TrainingArguments(
    output_dir=f'{pretrained_model_name}-finetuned-manual',
    overwrite_output_dir = True,
    metric_for_best_model='f1',
    greater_is_better=True,
    load_best_model_at_end=True,
    save_total_limit=4, 
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to="wandb",  # Enable logging to Weights & Biases
    run_name=f"{pretrained_model_name}-finetune-manual",  # Optionally set a specific run name    
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_ratio=0.2,
    num_train_epochs=2,
    lr_scheduler_type = 'linear',
)

trainer = QuestionAnsweringTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        eval_examples=eval_examples,
        tokenizer=tokenizer,
        data_collator=data_collator,
        post_process_function=post_processing_function,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

trainer.train()

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Exact,F1,Total,Hasans Exact,Hasans F1,Hasans Total,Noans Exact,Noans F1,Noans Total,Best Exact,Best Exact Thresh,Best F1,Best F1 Thresh
1,1.349600,No log,51.975069,60.428407,11873,24.358974,41.289892,5928,79.512195,79.512195,5945,53.154215,0.000000,60.634282,0.000000
2,1.013600,No log,49.473596,62.655210,11873,35.779352,62.180382,5928,63.128680,63.128680,5945,50.678009,0.000000,62.719799,0.000000


  0%|          | 0/11873 [00:00<?, ?it/s]

  0%|          | 0/11873 [00:00<?, ?it/s]

TrainOutput(global_step=8130, training_loss=1.5238689802669512, metrics={'train_runtime': 1923.7479, 'train_samples_per_second': 135.204, 'train_steps_per_second': 4.226, 'total_flos': 3.3982610012123136e+16, 'train_loss': 1.5238689802669512, 'epoch': 2.0})

## 5. Models Tuning Reusable Functions

In [53]:
Sample_Size = 60000

# Load a sample portion of the dataset
subset_squad_raw = datasets.DatasetDict({
    'train': cleaned_training_dataset.shuffle(seed=42).select(range(Sample_Size)),
    'validation': full_squad_dataset['validation']
})
    
# Display the sizes of the splits to confirm
print("Train set size:", len(subset_squad_raw['train']))
print("Validation set size:", len(subset_squad_raw['validation']))
squad_raw

# Preprocessing the datasets
eval_examples =  subset_squad_raw["validation"]


Train set size: 60000
Validation set size: 11873


In [54]:
class AdvancedEarlyStoppingCallback(TrainerCallback):
    """
    A callback to stop training when either the performance falls below a certain threshold
    or if there is no improvement over a set number of epochs.
    """
    def __init__(self, metric_name, patience, threshold):
        self.metric_name = metric_name
        self.patience = patience
        self.threshold = threshold
        self.best_score = None
        self.no_improve_epochs = 0

    def on_evaluate(self, args, state, control, **kwargs):
        metric_value = kwargs['metrics'].get(self.metric_name)

        if self.best_score is None or metric_value > self.best_score:
            self.best_score = metric_value
            self.no_improve_epochs = 0
        else:
            self.no_improve_epochs += 1

        # Check if performance is below the threshold
        if metric_value < self.threshold:
            control.should_training_stop = True
            print(f"Stopping training: {self.metric_name} below threshold of {self.threshold}")

        # Check if no improvement has been seen over the allowed patience
        if self.no_improve_epochs >= self.patience:
            control.should_training_stop = True
            print(f"Stopping training: No improvement in {self.metric_name} for {self.patience} epochs")


In [55]:
# Define model initialization function
def model_init():
    return AutoModelForQuestionAnswering.from_pretrained(pretrained_model_name)

# Define train dataset initialization function
def train_dataset_init():
    return subset_squad_raw['train'].map(
                preprocess_function,
                batched=True,
                remove_columns=subset_squad_raw["train"].column_names,
                desc="Running tokenizer on train dataset",
            )

# Define validation dataset initialization function
def vald_dataset_init():
    return subset_squad_raw['validation'].map(
                prepare_validation_features,
                batched=True,
                remove_columns=subset_squad_raw["train"].column_names,
                desc="Running tokenizer on validation dataset",
            )

# Optuna objective function for hyperparameter tuning
def objective(trial):
    # Hyperparameters to tune 
    global no_answer_threshold
    no_answer_threshold = trial.suggest_float('no_answer_threshold', 0.0, 1.0)
    #global global_doc_stride
    #global_doc_stride=trial.suggest_int('doc_stride', 128, 256, step=64)

    learning_rate = trial.suggest_float('learning_rate', 1e-7, 1e-4, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    #warmup_steps = trial.suggest_int('warmup_steps', 0, 1000)
    warmup_ratio= trial.suggest_int('warmup_ratio', 0.0, 1.0)
    weight_decay = trial.suggest_float('weight_decay', 0.0, 0.25)
    adam_beta1 = trial.suggest_float('adam_beta1', 0.8, 0.95)
    adam_beta2 = trial.suggest_float('adam_beta2', 0.990, 0.999)
    adam_epsilon = trial.suggest_float('adam_epsilon', 1e-8, 1e-6)
    lr_scheduler_type = trial.suggest_categorical('lr_scheduler_type', ['linear', 'cosine', 'cosine_with_restarts','constant_with_warmup'])
    output_dir = f"./{normalized_model_name}-finetuned-squadv2/trial_{trial.number}"
    
    # Print trial parameters
    print(f"Current Trial {trial.number} parameters: {trial.params}")
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir = True,
        metric_for_best_model='f1',
        greater_is_better=True,
        load_best_model_at_end=True,
        save_total_limit=2, # Save only the best model unless you specify a different number
        eval_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=3,  # Adjust based on computation limits
        report_to="wandb",  # Enable logging to Weights & Biases        
        run_name=f"{normalized_model_name}-finetune-squadv2",  # Optionally set a specific run name    
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        #warmup_steps=warmup_steps,
        warmup_ratio=warmup_ratio,
        weight_decay=weight_decay,
        adam_beta1=adam_beta1,
        adam_beta2=adam_beta2,
        adam_epsilon=adam_epsilon,
        lr_scheduler_type=lr_scheduler_type,
        fp16=True,  # Enable mixed-precision training
    )    

    trainer = QuestionAnsweringTrainer(
        model=model_init(),
        tokenizer=tokenizer,
        args=training_args,
        train_dataset=train_dataset_init(),
        eval_dataset=vald_dataset_init(),
        eval_examples=eval_examples,        
        data_collator=data_collator,
        post_process_function=post_processing_function,
        compute_metrics=compute_metrics,
        callbacks=[AdvancedEarlyStoppingCallback(metric_name='eval_f1', patience=1, threshold=60)]
    )  
    

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()
    #print("Evaluation results:", eval_results)  # Debug print
    return eval_results['eval_f1']


## 6. Hyperparameters Search for DistilBERT Model


In [90]:
# Create a study object and optimize the objective
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2024-05-28 13:10:08,274] A new study created in memory with name: no-name-4ccd1933-c04e-441c-8dc0-0b8436ab8c8f


Current Trial 0 parameters: {'no_answer_threshold': 0.4432355436688049, 'learning_rate': 1.9504510196346785e-06, 'batch_size': 64, 'warmup_ratio': 1, 'weight_decay': 0.22591500798452963, 'adam_beta1': 0.8295132212584269, 'adam_beta2': 0.9959028739768075, 'adam_epsilon': 6.79080552716296e-07, 'lr_scheduler_type': 'cosine_with_restarts'}


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Exact,F1,Total,Hasans Exact,Hasans F1,Hasans Total,Noans Exact,Noans F1,Noans Total,Best Exact,Best Exact Thresh,Best F1,Best F1 Thresh
1,6.226400,No log,0.732755,5.314734,11873,1.467611,10.644709,5928,0.000000,0.000000,5945,50.071591,0.000000,50.074654,0.000000


  0%|          | 0/11873 [00:00<?, ?it/s]

...

  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 60


  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 60
Stopping training: No improvement in eval_f1 for 1 epochs


[I 2024-05-28 17:30:42,075] Trial 19 finished with value: 54.07179634499088 and parameters: {'no_answer_threshold': 0.5481869225649895, 'learning_rate': 3.4187100043230674e-05, 'batch_size': 32, 'warmup_ratio': 0, 'weight_decay': 0.16531819100151934, 'adam_beta1': 0.8739210589706451, 'adam_beta2': 0.9910858068337787, 'adam_epsilon': 3.558108625151874e-07, 'lr_scheduler_type': 'linear'}. Best is trial 14 with value: 55.6012883577199.


In [97]:
study.optimize(objective, n_trials=5)

Current Trial 20 parameters: {'no_answer_threshold': 0.32828806396895294, 'learning_rate': 1.5375571315885884e-05, 'batch_size': 32, 'warmup_ratio': 0, 'weight_decay': 0.09230451414867091, 'adam_beta1': 0.8878529212131444, 'adam_beta2': 0.9914305059519523, 'adam_epsilon': 1.658544618504578e-07, 'lr_scheduler_type': 'linear'}


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Exact,F1,Total,Hasans Exact,Hasans F1,Hasans Total,Noans Exact,Noans F1,Noans Total,Best Exact,Best Exact Thresh,Best F1,Best F1 Thresh
1,1.875100,No log,42.491367,53.342933,11873,28.812416,50.546667,5928,56.131203,56.131203,5945,50.096858,0.000000,54.173886,0.000000


  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 60


...

  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 60


  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 60
Stopping training: No improvement in eval_f1 for 1 epochs


[I 2024-05-28 18:14:41,200] Trial 24 finished with value: 57.62459280679871 and parameters: {'no_answer_threshold': 0.5348322812842748, 'learning_rate': 5.3930827109666226e-05, 'batch_size': 32, 'warmup_ratio': 0, 'weight_decay': 0.220721938547819, 'adam_beta1': 0.9171076206148331, 'adam_beta2': 0.9946127382910206, 'adam_epsilon': 4.517836581754784e-07, 'lr_scheduler_type': 'linear'}. Best is trial 24 with value: 57.62459280679871.


In [99]:
study.optimize(objective, n_trials=5)

Current Trial 25 parameters: {'no_answer_threshold': 0.33385240306636493, 'learning_rate': 1.29424009940198e-05, 'batch_size': 32, 'warmup_ratio': 0, 'weight_decay': 0.21686724890909415, 'adam_beta1': 0.9184527601580457, 'adam_beta2': 0.9948430917566237, 'adam_epsilon': 4.5372239250548926e-07, 'lr_scheduler_type': 'linear'}


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Exact,F1,Total,Hasans Exact,Hasans F1,Hasans Total,Noans Exact,Noans F1,Noans Total,Best Exact,Best Exact Thresh,Best F1,Best F1 Thresh
1,1.950000,No log,39.400320,50.502679,11873,29.065452,51.302009,5928,49.705635,49.705635,5945,50.096858,0.000000,51.557021,0.000000


  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 60


...

  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 60


  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 60
Stopping training: No improvement in eval_f1 for 1 epochs


[I 2024-05-28 19:50:02,319] Trial 29 finished with value: 50.07159100480081 and parameters: {'no_answer_threshold': 0.40133773868646083, 'learning_rate': 1.4027443018268331e-06, 'batch_size': 64, 'warmup_ratio': 0, 'weight_decay': 0.22949744991020696, 'adam_beta1': 0.8861148162582255, 'adam_beta2': 0.9957144331116927, 'adam_epsilon': 1.7765035688797953e-07, 'lr_scheduler_type': 'linear'}. Best is trial 24 with value: 57.62459280679871.


In [101]:
study.optimize(objective, n_trials=5)

Current Trial 30 parameters: {'no_answer_threshold': 0.1176376945719293, 'learning_rate': 5.139828322033379e-05, 'batch_size': 64, 'warmup_ratio': 0, 'weight_decay': 0.16115642901725802, 'adam_beta1': 0.9337762788645813, 'adam_beta2': 0.9931344908814995, 'adam_epsilon': 4.318766340760206e-07, 'lr_scheduler_type': 'linear'}


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Exact,F1,Total,Hasans Exact,Hasans F1,Hasans Total,Noans Exact,Noans F1,Noans Total,Best Exact,Best Exact Thresh,Best F1,Best F1 Thresh
1,2.270600,No log,46.668913,58.470220,11873,31.578947,55.215404,5928,61.715728,61.715728,5945,50.096858,0.000000,58.991977,0.000000


  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 60


...

  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 60


  0%|          | 0/11873 [00:00<?, ?it/s]

Stopping training: eval_f1 below threshold of 60
Stopping training: No improvement in eval_f1 for 1 epochs


[I 2024-05-28 22:52:55,644] Trial 34 finished with value: 46.893566098021374 and parameters: {'no_answer_threshold': 0.0840898747658837, 'learning_rate': 2.157833848624328e-05, 'batch_size': 64, 'warmup_ratio': 1, 'weight_decay': 0.13303340459210167, 'adam_beta1': 0.9497676255380477, 'adam_beta2': 0.9929853840179018, 'adam_epsilon': 5.436707707276962e-07, 'lr_scheduler_type': 'cosine'}. Best is trial 30 with value: 58.47021961070583.


## 7. Summary of Hyperparameters Search

Each row of the below table corresponds to a trial and shows all specified parameters along with measured performance metrics with Trial 9 acheiving the highest performance.



| Trial | Learning Rate     | Batch Size | Warmup Steps | Weight Decay   | Adam Beta1 | Adam Beta2 | Adam Epsilon | LR Scheduler Type          | Exact   | F1       | Hasans Exact | Hasans F1 | Noans Exact | Noans F1 |
|-------|-------------------|------------|--------------|----------------|------------|------------|--------------|----------------------------|---------|----------|--------------|-----------|-------------|----------|


## Tuning DistilBERT Using the Best Hyperparameters

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(pretrained_model_name)
training_args = TrainingArguments(
    output_dir=f"./{normalized_model_name}-best_model",
    overwrite_output_dir = True,
    metric_for_best_model='f1',
    greater_is_better=True,
    load_best_model_at_end=True,
    save_total_limit=4, 
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    report_to="wandb",  # Enable logging to Weights & Biases
    run_name=f"{normalized_model_name}-best_model",
    learning_rate=4.91978224351371e-05,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=398,
    weight_decay=0.19404494917360213,
    adam_beta1=0.8368183686077211,
    adam_beta2=0.9969107098230887,
    adam_epsilon=5.874878468800929e-07,
    lr_scheduler_type='cosine',
    fp16=True,  # Enable mixed-precision training
)

trainer = QuestionAnsweringTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        eval_examples=eval_examples,
        tokenizer=tokenizer,
        data_collator=data_collator,
        post_process_function=post_processing_function,
        compute_metrics=compute_metrics,
        callbacks=[AdvancedEarlyStoppingCallback(metric_name='eval_f1', patience=3, threshold=40)]
)

trainer.train()

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Exact,F1,Total,Hasans Exact,Hasans F1,Hasans Total,Noans Exact,Noans F1,Noans Total,Best Exact,Best Exact Thresh,Best F1,Best F1 Thresh
1,1.262600,No log,52.690980,63.392637,11873,30.364372,51.798377,5928,74.953743,74.953743,5945,53.373200,0.000000,63.405070,0.000000
2,0.962700,No log,51.217047,65.262790,11873,37.348178,65.479943,5928,65.046257,65.046257,5945,51.486566,0.000000,65.262790,0.000000
3,0.713500,No log,49.953676,64.202782,11873,37.736167,66.275242,5928,62.136249,62.136249,5945,50.509560,0.000000,64.211205,0.000000
4,0.526100,No log,48.825065,63.751619,11873,38.107287,68.003201,5928,59.512195,59.512195,5945,50.290575,0.000000,63.760042,0.000000


  0%|          | 0/11873 [00:00<?, ?it/s]

  0%|          | 0/11873 [00:00<?, ?it/s]

  0%|          | 0/11873 [00:00<?, ?it/s]

  0%|          | 0/11873 [00:00<?, ?it/s]